# 決定木

## 参考資料

* https://www.kaggle.com/code/antonoof/decisiontree-numpy-simple-code
* https://qiita.com/3000manJPY/items/ef7495960f472ec14377
* https://zenn.dev/mi_01_24fu/articles/regression-tree#%E5%90%84%E3%83%91%E3%82%BF%E3%83%BC%E3%83%B3%E3%81%A7sse%E3%82%92%E7%AE%97%E5%87%BA%E3%81%97%E3%81%9F%E7%B5%90%E6%9E%9C%E3%80%81%E6%9C%80%E5%B0%8Fsse%E3%81%8C%E5%88%86%E3%81%8B%E3%81%A3%E3%81%9F%E3%80%81%E3%81%98%E3%82%83%E3%81%82%E3%81%A9%E3%81%86%E3%81%99%E3%82%8B%E3%81%AE%EF%BC%9F

## 理論

決定木とは分類や数値予測(回帰)を行う教師あり機械学習手法の一つ。
分類木と回帰木をまとめて決定木と呼ぶ。 <br><br>
分類木 : 与えられた特徴量に基づき、対象データがどのカテゴリに属するか判断するモデル <br>
ex) メールを「スパム」「非スパム」に分類する。 <br><br>
回帰木 : 連続的な値の予測に特化したモデル <br>
ex) 過去の地球温暖化のデータを利用して将来の気温上昇を予測する。

分岐させる時の閾値は人間が決めるのではなく、自動で最適化する。
そのとき、データの持つ特徴の中で集められたデータを一番よく分割する特徴とその閾値の組を選ぶことが肝になる。
この組を選び出すのによく使われているのが「エントロピー」と「ジニ不純度」である。

### エントロピー

定義

$$ H = - \sum_{i = 1}^n p_i \log p_i $$

$ p_i $はクラス$ i $に属する確率(割合)を表すので、$ p_i = \dfrac{n_i}{N} $と書くことができる。 <br>
$ \log $の底はなんでも良いが、クラス数$ n $にしておくと、エントロピーの最大値が$ 1 $となる。

エントロピーはデータの不純度が大きいほど大きくなり、データの不純度が小さいほど小さくなる。 <br>
したがってデータの不純度度合いをエントロピーを用いて数値化できる。

では、どのようにしてデータの分割の良し悪しを判断するのだろうか。 <br>
そこで利得(gain)をエントロピーを用いる。

$$ \Delta H(t) = H(t_B) - \sum_{i = 1}^b w_i H_i(t_{Ai}) $$

$ b $ : 分岐(ブランチ)の数 <br>
$ t_B $ : 分岐前のデータ　<br>
$ t_A $ : 分岐後のデータ <br>
$ w_i $ : 重み(分岐前に対するデータの量の割合)

$$ \text{(分岐前のエントロピー)} - \sum \text{重み} \times \text{(分岐後のエントロピー)} $$

分岐した時にうまく不純度が小さくなっていれば、右辺の分岐後のエントロピーが小さくなり、利得が大きくなる。 <br>
つまり、この式の計算結果が最大となるような特徴と閾値の組が不純度を最小にする組であるとわかる。 <br>
この分割指標を用いて決定木を構築していくアルゴリズムがC4.5である。

### ジニ不純度

定義

$$ G = 1 - \sum_{i = 1}^n p_i^2 $$

ジニ不純度もエントロピーと同様、データの不純度が大きいほど値が大きくなり、データが不純度が小さいほど値が小さくなる。<br>
また、不純度が最も低ければジニ不純度の値は$ 0 $、不純度が大きくなるとジニ不純度の値が$ 1 $に近づく。(最大でも$ 1 - \dfrac{1}{n}$なので$ 1 $は取りえない)

エントロピーと同様に利得を計算することで、データの分割の良し悪しを判断する。

$$ \Delta G = G(t_B) - w_L G(t_L) - w_R G(t_R) $$

$ t_B $ : 分岐前のノード <br>
$ t_L, t_R $ : 分岐後の左(右)ノード

不純度がうまく分岐によって低くなると、利得は大きくなる。 <br>
よって、ジニ不純度の場合も値を最大にする特徴と閾値の組が、最も不純度を抑える組である。

この分割指標を用いて決定木を構築していくアルゴリズムがCARTである。

## 実装

### 準備

In [8]:
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

データの読み込み

In [3]:
train = pd.read_csv("../data/input/train.csv")
test = pd.read_csv("../data/input/test.csv")
submission = pd.read_csv("../data/input/sample_submission.csv")

`LabelEncoder()`を用いて文字列をアルファベット順に番号づけする。

In [42]:
le = LabelEncoder()
train["Personality_encoded"] = le.fit_transform(train["Personality"])

`drop`を使用して`id`から`Personality`を除いたデータフレームを作成

In [10]:
X = train.drop(columns=["id", "Personality", "Personality_encoded"])
y = train["Personality_encoded"]

数値的な特徴量は平均で欠損値を埋め、カテゴリカルな特徴量は最頻値で埋める。

In [27]:
numerical_features = X.select_dtypes(include=['number']).columns
categorical_cols = X.select_dtypes(exclude=['number']).columns

X[numerical_features] = X[numerical_features].fillna(X[numerical_features].mean())

for col in categorical_cols:
    if X[col].isnull().any():
        X[col] = X[col].fillna(X[col].mode()[0])

カテゴリカルな特徴量を数値に変換する。

In [28]:
for col in categorical_cols:
    X[col] = le.fit_transform(X[col])

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [30]:
print("X_train", X_train)
print("y_train", y_train)

X_train        Time_spent_Alone  Stage_fear  Social_event_attendance  Going_outside  \
18072               1.0           0                      8.0       4.000000   
11365               3.0           0                      9.0       3.000000   
3470                8.0           1                      1.0       3.000000   
7242                2.0           0                      4.0       7.000000   
1281                8.0           1                      1.0       0.000000   
...                 ...         ...                      ...            ...   
17592               3.0           0                      7.0       6.000000   
17240              10.0           1                      1.0       1.000000   
6043                1.0           0                      7.0       5.000000   
5605                1.0           0                      6.0       5.000000   
13373               3.0           0                      7.0       4.044319   

       Drained_after_socializing  Friends_c

### 決定木クラスの実装

In [ ]:
# Tree Node
# left(right) : left(right) node
class TreeNode:
    def __init__(self, feature=None, threshold=None, val=None, left=None, right=None):
        self.feature = feature
        self.threshold = threshold
        self.val = val
        self.left = left
        self.right = right

    def __str__(self):
        left_str = str(self.left) if self.left else "None"
        right_str = str(self.right) if self.right else "None"

        return f"TreeNode(val={self.val}, feature={self.feature}, threshold={self.threshold}, left={left_str}, right={right_str})"

クラスのテスト

In [16]:
test_node = TreeNode()

test_node.val = 1
test_node.left = TreeNode(val=2, left=3, right=4)

print(test_node)

TreeNode(val=1, feature=None, threshold=None, left=TreeNode(val=2, feature=None, threshold=None, left=3, right=4), right=None)


In [19]:
class MyDecisionTreeClassifier:

    def __init__(self, data, target):
        self.data = data
        self.target = target
        self.Tree = None

    def gini_impurity(self, y):
        ##### Calculate Gini impurity #####
        total = len(y)

        if total == 0:
            return 0

        counts = Counter(y)
        impurity = 1 - sum((count / total) ** 2 for count in counts.values())

        return impurity

    def best_split(self, X, y):
        ##### Find the best feature and threshold to split the data #####
        best_gini = float("inf")
        best_feature = None
        best_threshold = None
        n_features = X.shape[1]

        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices = X[:, feature] < threshold
                right_indices = X[:, feature] >= threshold

                if np.any(left_indices) and np.any(right_indices):
                    left_classes = y[left_indices]
                    right_classes = y[right_indices]

                    gini_left = self.gini_impurity(left_classes)
                    gini_right = self.gini_impurity(right_classes)
                    weighted_gini = (len(left_classes) / len(y) * gini_left + len(right_classes) / len(y) * gini_right) / len(y)

                    if weighted_gini < best_gini:
                        best_gini = weighted_gini
                        best_feature = feature
                        best_threshold = threshold

        return best_feature, best_threshold

    def build_tree(self, X, y):
        ##### Recursively build the decision tree #####
        if len(set(y)) == 1:
            return TreeNode(val=y[0])

        feature, threshold = self.best_split(X, y)

        if feature is None:
            return TreeNode(val=Counter(y).most_common(1)[0][0])

        left_indices = X[:, feature] < threshold
        right_indices = X[:, feature] >= threshold

        left_node = self.build_tree(X[left_indices], y[left_indices])
        right_node = self.build_tree(X[right_indices], y[right_indices])

        return TreeNode(feature=feature, threshold=threshold, left=left_node, right=right_node)

    def fit(self):
        ##### Fit the decision tree to the training data #####
        self.Tree = self.build_tree(self.data, self.target)

    def predict_one(self, node, x):
        ##### Predict the class for a single sample #####
        if node.val is not None:
            return node.val

        if x[node.feature] < node.threshold:
            return self.predict_one(node.left, x)
        else:
            return self.predict_one(node.right, x)

    def predict(self, X):
        ##### Predict the class for multiple samples #####
        return [self.predict_one(self.Tree, x) for x in X]


### 実際に使用する

In [33]:
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

my_tree = MyDecisionTreeClassifier(X_train_np, y_train_np)
my_tree.fit()
print("Training complete.")
print(my_tree.Tree)

Training complete.
TreeNode(val=None, feature=4, threshold=1.0, left=TreeNode(val=None, feature=1, threshold=1.0, left=TreeNode(val=None, feature=0, threshold=7.0, left=TreeNode(val=None, feature=5, threshold=4.0, left=TreeNode(val=None, feature=2, threshold=4.0, left=TreeNode(val=None, feature=3, threshold=5.0, left=TreeNode(val=None, feature=2, threshold=3.0, left=TreeNode(val=None, feature=0, threshold=2.0, left=TreeNode(val=1, feature=None, threshold=None, left=None, right=None), right=TreeNode(val=None, feature=0, threshold=4.0, left=TreeNode(val=0, feature=None, threshold=None, left=None, right=None), right=TreeNode(val=None, feature=5, threshold=3.0, left=TreeNode(val=1, feature=None, threshold=None, left=None, right=None), right=TreeNode(val=0, feature=None, threshold=None, left=None, right=None)))), right=TreeNode(val=1, feature=None, threshold=None, left=None, right=None)), right=TreeNode(val=None, feature=5, threshold=2.0, left=TreeNode(val=None, feature=0, threshold=5.0, le

In [32]:
X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()

predictions = my_tree.predict(X_test_np)

accuracy = accuracy_score(y_test_np, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9225371120107962


In [34]:
print("predictions", predictions)

predictions [np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0),

## テストデータを使用して予測＆提出

In [38]:
numerical_features_test = test.select_dtypes(include=['number']).columns
categorical_cols_test = test.select_dtypes(exclude=['number']).columns

test[numerical_features_test] = test[numerical_features_test].fillna(test[numerical_features_test].mean())

for col in categorical_cols_test:
    if test[col].isnull().any():
        test[col] = test[col].fillna(test[col].mode()[0])

for col in categorical_cols_test:
    test[col] = le.fit_transform(test[col])

test.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.00000,0,7.0,4.0,0,6.0,5.028958
1,18525,3.11687,1,0.0,0.0,1,5.0,1.000000
2,18526,3.00000,0,5.0,6.0,0,15.0,9.000000
3,18527,3.00000,0,4.0,4.0,0,5.0,6.000000
4,18528,9.00000,1,1.0,2.0,1,1.0,1.000000


In [43]:
test_predictions = my_tree.predict(test.drop(columns=["id"]).to_numpy())
final_predictions = np.array(test_predictions).astype(int)

submission["Personality"] = le.inverse_transform(final_predictions)
submission.head()

,id,Personality
0,18524,Extrovert
1,18525,Introvert
2,18526,Extrovert
3,18527,Extrovert
4,18528,Introvert


In [44]:
submission.to_csv("../data/output/submission.csv", index=False)
print("Submission saved to ../data/output/submission.csv")

Submission saved to ../data/output/submission.csv


csvを提出する

In [49]:
!kaggle competitions submit -c playground-series-s5e7 -f ../data/output/submission.csv -m "My Decision Tree Classifier"

100%|██████████████████████████████████████| 96.5k/96.5k [00:01<00:00, 95.2kB/s]
Successfully submitted to Predict the Introverts from the Extroverts